# Comparing Geometries

In [523]:
import scipy
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
import math
from scipy.optimize import leastsq
method_dict = {'PM3':-7,'AM1':-2, 'RM1':-2, 'OM1':-5, 'OM2':-6, 'OM3':-8,'ODM2':-22, 'ODM3':-23}
            
at_num_dict = {'h':1, 'he':2, 'li':3, 'be':4, 'b':5,'c':6, 'n':7, 'o':8, 'f':9}
def read_input(file):
    with open(file) as f_in:
        lines = f_in.readlines() 
    n_parms= int(lines[0].split()[0])
    method = lines[1].split()[0]
    method_num = method_dict[method.upper().replace("ORTHO", "")]
    n_molec = int(lines[2].split()[0])
    n_atoms=[]
    charge =[]
    structures=[]
    energy_files=[]
    structure_files = []
    at_num=[[] for x in range(n_molec)]
    coords=[[] for x in range(n_molec)]
    geoms =[[] for x in range(n_molec)]
    
    
    n = 3
    for mol in range(n_molec):
        n_atoms.append(int(lines[n].split()[0]))   # set the number of atoms in a molecule
        charge.append(int(lines[n+1].split()[0]))  # set the charge on the molecule
        structures.append(int(lines[n+2].split()[0])) # set the number of structures
        energy_files.append(lines[n+3].split()[0])  # set the file to find the energies
        structure_files.append([lines[n+4].split()[0],int(lines[n+4].split()[1])]) # set the filename for the structures (the second flag is for atomic units = 1 or angstroms =0)
        if (structure_files[mol][1] == 1):
            scale = 0.529
        else:
            scale = 1.
        for atom in range(n_atoms[mol]):
            line = lines[n+5+atom].split() 
            check = np.array(line[1:],dtype=float)
            at_num[mol].append(at_num_dict[line[0].lower()])
            coords[mol].append(np.array([x*scale for x in check]))
        n += n_atoms[mol]+5
    return method_num, n_molec, n_atoms, charge, structures, energy_files, structure_files, at_num, coords


In [524]:
meth_num, n_molec, n_atoms, charge, structures, energy_files, structure_files, at_num, coords = read_input("main.inp")

In [525]:
outfile = open("opt0.out", "r")
optlines = outfile.readlines()
intgeom = []
optgeom = []
for n,line in enumerate(optlines):
    if "INPUT GEOMETRY" in line:
        for atoms in range(n_atoms[0]):
            m = optlines[n+atoms+6].strip()
            o = m.split()[2::2]
            intgeom.append(o)
    if "FINAL CARTESIAN GRADIENT NORM" in line:
        for atoms in range(n_atoms[0]):
            z = optlines[n+atoms+8].strip()
            y = z.split()[2::2]
            optgeom.append(y)
intgeom = np.array(intgeom).astype(float)
optgeom = np.array(optgeom).astype(float)


readfile = open("main.inp", "r")
lines = readfile.readlines()
text_file = open("geom-exmpl.out", "w")
for line in lines: 
    if "bond" in line:
        l = line.split()
        at_bond = ' '.join(map(str, l))
        dist1 = np.linalg.norm(coords[0][int(l[1])-1]-coords[0][int(l[2])-1])
        dist4 = np.linalg.norm(optgeom[int(l[1])-1]-optgeom[int(l[2])-1])
        diff1 = dist1-dist4
        print(f'{at_bond} {dist1:.4} {dist4:.4} {diff1:.4}', file=text_file) #how many decimal places?
    if "angle" in line: #more direct way to do this?
        m = line.split()    
        at_ang = ' '.join(map(str, m))
        dot1 = np.dot((coords[0][int(m[1])-1]-coords[0][int(m[2])-1]), (coords[0][int(m[3])-1]-coords[0][int(m[2])-1]))
        dist2 = np.linalg.norm(coords[0][int(m[1])-1]-coords[0][int(m[2])-1])
        dist3 = np.linalg.norm(coords[0][int(m[3])-1]-coords[0][int(m[2])-1])
        cos1 = dot1/(dist2*dist3)
        angle1 = (math.acos(cos1))*57.295779513
        dot2 = np.dot((optgeom[int(m[1])-1]-optgeom[int(m[2])-1]), (optgeom[int(m[3])-1]-optgeom[int(m[2])-1]))
        dist5 = np.linalg.norm(optgeom[int(m[1])-1]-optgeom[int(m[2])-1])
        dist6 = np.linalg.norm(optgeom[int(m[3])-1]-optgeom[int(m[2])-1])
        cos2 = dot2/(dist5*dist6)
        angle2 = (math.acos(cos2))*57.295779513
        diff2 = angle1-angle2
        print(f'{at_ang} {angle1:.4} {angle2:.4} {diff2:.4}', file=text_file)
outfile.close()
readfile.close()


In [529]:
text_file = open("geom-exmpl.out", "w")
geoms = [[['bond', '1', '2'], ['bond', '2', '4'], ['angle', '3', '1', '2'], ['angle', '1', '3', '5']]]
for mol in range(n_molec):
    for geom in geoms[mol]:
        if geom[0] == 'bond':
            at_bond = ' '.join(map(str, geom))
            dist1 = np.linalg.norm(intgeom[int(geom[1])-1]-intgeom[int(geom[2])-1])
            dist4 = np.linalg.norm(optgeom[int(geom[1])-1]-optgeom[int(geom[2])-1])
            diff1 = dist1-dist4
            print(f'{at_bond} {dist1:.4} {dist4:.4} {diff1:.4}', file=text_file) #how many decimal places?
        if geom[0] == 'angle':    
            at_ang = ' '.join(map(str, geom))
            dot1 = np.dot((intgeom[int(geom[1])-1]-intgeom[int(geom[2])-1]), (intgeom[int(geom[3])-1]-intgeom[int(geom[2])-1]))
            dist2 = np.linalg.norm(intgeom[int(geom[1])-1]-intgeom[int(geom[2])-1])
            dist3 = np.linalg.norm(intgeom[int(geom[3])-1]-intgeom[int(geom[2])-1])
            cos1 = dot1/(dist2*dist3)
            angle1 = (math.acos(cos1))*57.295779513
            dot2 = np.dot((optgeom[int(geom[1])-1]-optgeom[int(geom[2])-1]), (optgeom[int(geom[3])-1]-optgeom[int(geom[2])-1]))
            dist5 = np.linalg.norm(optgeom[int(geom[1])-1]-optgeom[int(geom[2])-1])
            dist6 = np.linalg.norm(optgeom[int(geom[3])-1]-optgeom[int(geom[2])-1])
            cos2 = dot2/(dist5*dist6)
            angle2 = (math.acos(cos2))*57.295779513
            diff2 = angle1-angle2
            print(at_ang)
            print(f'{at_ang} {angle1:.4} {angle2:.4} {diff2:.4}', file=text_file)
text_file.close()

angle 3 1 2
angle 1 3 5


# Checking the Answer is Correct with Math by "Hand"

In [304]:
dist0 = np.linalg.norm(coords[0][0]-coords[0][1])

In [22]:
np.sqrt(((-1.67486-.69388)**2+1.24986**2))*0.529

1.416799794756498

Checked the angle math with a website

In [530]:
np.

Signature: np.vstack(tup)
Docstring:
Stack arrays in sequence vertically (row wise).

This is equivalent to concatenation along the first axis after 1-D arrays
of shape `(N,)` have been reshaped to `(1,N)`. Rebuilds arrays divided by
`vsplit`.

This function makes most sense for arrays with up to 3 dimensions. For
instance, for pixel-data with a height (first axis), width (second axis),
and r/g/b channels (third axis). The functions `concatenate`, `stack` and
`block` provide more general stacking and concatenation operations.

Parameters
----------
tup : sequence of ndarrays
    The arrays must have the same shape along all but the first axis.
    1-D arrays must have the same length.

Returns
-------
stacked : ndarray
    The array formed by stacking the given arrays, will be at least 2-D.

See Also
--------
stack : Join a sequence of arrays along a new axis.
hstack : Stack arrays in sequence horizontally (column wise).
dstack : Stack arrays in sequence depth wise (along third dimensi